## Student Name: 
## Student Email:

# Project 3: The Smart City Slicker

Imagine you are a stakeholder in a rising Smart City and want to know more about themes and concepts about existing smart cities. You also want to know where does your smart city place among others. In this project, you will perform 
exploratory data analysis, often shortened to EDA, to examine a data from the [2015 Smart City Challenge](https://www.transportation.gov/smartcity) to find facts about the data and communicating those facts through text analysis and visualizations.

In order to explore the data and visualize it, some modifications might need to be made to the data along the way. This is often referred to as data preprocessing or cleaning.
Though data preprocessing is technically different from EDA, EDA often exposes problems with the data that need to be fixed in order to continue exploring.
Because of this tight coupling, you have to clean the data as necessary to help understand the data.

In this project, you will apply your knowledge about data cleaning, machine learning, visualizations, and databases to explore smart city applications.

**Part 1** of the notebook will explore and clean the data. \
**Part 2** will take the results of the preprocessed data to create models and visualizations.

Empty cells are code cells. 
Cells denoted with [Your Answer Here] are markdown cells.
Edit and add as many cells as needed.

Output file for this notebook is shown as a table for display purposes. Note: The city name can be Norman, OK or OK Norman.

| city | raw text | clean text | clusterid | topicids | 
| -- | -- | -- | -- | -- | 
|Norman, OK | Test, test , and testing. | test test test | 0 | T1, T2| 

## Introduction
The Dataset: 2015 Smart City Challenge Applicants (non-finalist).
In this project you will use the applicant's PDFs as a dataset.
The dataset is from the U.S Department of Transportation Smart City Challenge.

On the website page for the data, you can find some basic information about the challenge. This is an interesting dataset. Think of the questions that you might be able to answer! A few could be:

1. Can I identify frequently occurring words that could be removed during data preprocessing?
2. Where are the applicants from?
3. Are there multiple entries for the same city in different applicantions?
4. What are the major themes and concepts from the smart city applicants?

Let's load the data!

## Loading and Handling files (Required)

Load data from `smartcity/`. 

To extract the data from the pdf files, use the [pypdf.pdf.PdfFileReader](https://pypdf.readthedocs.io/en/stable/index.html) class.
It will allow you to extract pages and pdf files and add them to a data structure (dataframe, list, dictionary, etc).
To install the module, use the command `pipenv install pypdf`.
You only need to handle PDF files, handling docx is not necessary.

In [13]:
import PyPDF2
import os


directory = 'smartcity'
files_data = {}

for filename in os.listdir(directory):
    if filename.endswith(".pdf"):
        filepath = os.path.join(directory, filename)
        with open(filepath, 'rb') as pdfFileObj:
            pdfReader = PyPDF2.PdfReader(pdfFileObj)
            num_pages = len(pdfReader.pages)
            text = ""
            for i in range(num_pages):
                page = pdfReader.pages[i]
                page_text = page.extract_text()
                page_lines = page_text.splitlines()
                text += page_text
            files_data[filename] = text
            print(filename)
            continue


GA Brookhaven.pdf
FL Tallahassee.pdf
NY Buffalo.pdf
CA Riverside.pdf
AZ Scottsdale AZ.pdf
FL Jacksonville.pdf
LA New Orleans.pdf
AL Montgomery.pdf
MI Port Huron and Marysville.pdf
WA Seattle.pdf
LA Shreveport.pdf
WA Spokane.pdf
IN Indianapolis.pdf
AL Birmingham.pdf
LA Baton Rouge.pdf
FL Miami.pdf
CA Oceanside.pdf


unknown widths : 
[0, IndirectObject(532, 0, 140435168573376)]
unknown widths : 
[0, IndirectObject(535, 0, 140435168573376)]
unknown widths : 
[0, IndirectObject(538, 0, 140435168573376)]
unknown widths : 
[0, IndirectObject(541, 0, 140435168573376)]
unknown widths : 
[0, IndirectObject(544, 0, 140435168573376)]
unknown widths : 
[0, IndirectObject(547, 0, 140435168573376)]
unknown widths : 
[0, IndirectObject(550, 0, 140435168573376)]
unknown widths : 
[0, IndirectObject(553, 0, 140435168573376)]


CA San Jose_0.pdf
NE Lincoln.pdf
MA Boston.pdf
CA Sacramento.pdf
VA Richmond.pdf
GA Atlanta.pdf
NY Rochester.pdf
TN Memphis.pdf
NC Raleigh.pdf
NY Albany Troy Schenectady Saratoga Springs.pdf
OH Cleveland.pdf
NC Charlotte.pdf
NJ Jersey City.pdf
CA Chula Vista.pdf
CA Long Beach.pdf
MI Detroit.pdf
IA Des Moines.pdf
MO St. Louis.pdf
NE Omaha.pdf
OH Akron.pdf
VA Newport News.pdf
NY Mt Vernon Yonkers New Rochelle.pdf
CA Fremont.pdf
MD Baltimore.pdf
SC Greenville.pdf
CT NewHaven.pdf
TX Lubbock.pdf
CA Fresno.pdf
CA Oakland.pdf
TN Chattanooga.pdf
RI Providence.pdf
AK Anchorage.pdf
AZ Tucson.pdf
MN Minneapolis St Paul.pdf
NV Reno.pdf
OH Toledo.pdf
NC Greensboro.pdf
OH Canton.pdf
NV Las Vegas.pdf
TN Nashville.pdf
OK Oklahoma City.pdf
WI Madison.pdf
NJ Newark.pdf
KY Louisville.pdf
FL St. Petersburg.pdf
CA Moreno Valley.pdf
FL Tampa.pdf
VA Norfolk.pdf
DC_0.pdf
FL Orlando.pdf
VA Virginia Beach.pdf
OK Tulsa.pdf


Create a data structure to add the city name and raw text. You can choose to split the city name from the file.

In [4]:
# Added it above


## Cleaning Up PDFs (Required)

One of the more frustrating aspects of PDF is loading the data into a readable format. The first order of business will be to preprocess the data. To start, you can use code provided by Text Analytics with Python, [Chapter 3](https://github.com/dipanjanS/text-analytics-with-python/blob/master/New-Second-Edition/Ch03%20-%20Processing%20and%20Understanding%20Text/Ch03a%20-%20Text%20Wrangling.ipynb): [contractions.py](https://github.com/dipanjanS/text-analytics-with-python/blob/master/New-Second-Edition/Ch05%20-%20Text%20Classification/contractions.py) (Pages 136-137), and [text_normalizer.py](https://github.com/dipanjanS/text-analytics-with-python/blob/master/New-Second-Edition/Ch05%20-%20Text%20Classification/text_normalizer.py) (Pages 155-156). Feel free to download the scripts or add the code directly to the notebook (please note this code is performed on dataframes).

In addition to the data cleaning provided by the textbook, you will need to:
1. Consider removing terms that may effect clustering and topic modeling. Words to consider are cities, states, common words (smart, city, page, etc.). Keep in mind n-gram combinations are important; this can also be revisited later depending on your model's performance.
2. Check the data to remove applicants that text was not processed correctly. Do not remove more than 15 cities from the data.


In [14]:
import nltk
import spacy
import unicodedata
from contractions import CONTRACTION_MAP
import re
from nltk.corpus import wordnet
import collections
#from textblob import Word
from nltk.tokenize.toktok import ToktokTokenizer
from bs4 import BeautifulSoup

nltk.download('stopwords')

tokenizer = ToktokTokenizer()
stopword_list = nltk.corpus.stopwords.words('english')
nlp = spacy.load('en_core_web_sm', exclude=['parser'])
# nlp_vec = spacy.load('en_vectors_web_lg', parse=True, tag=True, entity=True)

def strip_html_tags(text):
    soup = BeautifulSoup(text, "html.parser")
    if bool(soup.find()):
        [s.extract() for s in soup(['iframe', 'script'])]
        stripped_text = soup.get_text()
        stripped_text = re.sub(r'[\r|\n|\r\n]+', '\n', stripped_text)
    else:
        stripped_text = text
    return stripped_text


#def correct_spellings_textblob(tokens):
#	return [Word(token).correct() for token in tokens]  


def simple_porter_stemming(text):
    ps = nltk.porter.PorterStemmer()
    text = ' '.join([ps.stem(word) for word in text.split()])
    return text


def lemmatize_text(text):
    text = nlp(text)
    text = ' '.join([word.lemma_ if word.lemma_ != '-PRON-' else word.text for word in text])
    return text


def remove_repeated_characters(tokens):
    repeat_pattern = re.compile(r'(\w*)(\w)\2(\w*)')
    match_substitution = r'\1\2\3'
    def replace(old_word):
        if wordnet.synsets(old_word):
            return old_word
        new_word = repeat_pattern.sub(match_substitution, old_word)
        return replace(new_word) if new_word != old_word else new_word
            
    correct_tokens = [replace(word) for word in tokens]
    return correct_tokens


def expand_contractions(text, contraction_mapping=CONTRACTION_MAP):
    
    contractions_pattern = re.compile('({})'.format('|'.join(contraction_mapping.keys())), 
                                      flags=re.IGNORECASE|re.DOTALL)
    def expand_match(contraction):
        match = contraction.group(0)
        first_char = match[0]
        expanded_contraction = contraction_mapping.get(match)\
                                if contraction_mapping.get(match)\
                                else contraction_mapping.get(match.lower())                       
        expanded_contraction = first_char+expanded_contraction[1:]
        return expanded_contraction
        
    expanded_text = contractions_pattern.sub(expand_match, text)
    expanded_text = re.sub("'", "", expanded_text)
    return expanded_text


def remove_accented_chars(text):
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text


def remove_special_characters(text, remove_digits=False):
    pattern = r'[^a-zA-Z0-9\s]|\[|\]' if not remove_digits else r'[^a-zA-Z\s]|\[|\]'
    text = re.sub(pattern, '', text)
    return text


def remove_stopwords(text, is_lower_case=False, stopwords=stopword_list):
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stopwords]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stopwords]
    filtered_text = ' '.join(filtered_tokens)    
    return filtered_text

def remove_cities_states(text) :
    labels = ['GPE', 'LOC', 'PERSON', 'ORG']

    cities_states = ['Brookhaven', 'Tallahassee', 'Buffalo', 'Riverside', 'Scottsdale', 'Jacksonville', 'New Orleans', 'Montgomery', 'Port Huron', 'Marysville', 'Seattle', 'Shreveport', 'Spokane', 'Indianapolis', 'Birmingham', 'Baton Rouge', 'Miami', 'Oceanside', 'San Jose', 'Lincoln', 'Boston', 'Sacramento', 'Richmond', 'Atlanta', 'Rochester', 'Memphis', 'Raleigh', 'Albany', 'Troy', 'Schenectady', 'Saratoga Springs', 'Cleveland', 'Charlotte', 'Jersey City', 'Chula Vista', 'Long Beach', 'Detroit', 'Des Moines', 'St. Louis', 'Omaha', 'Akron', 'Newport News', 'Mt Vernon', 'Yonkers', 'New Rochelle', 'Fremont', 'Baltimore', 'Greenville', 'NewHaven', 'Lubbock', 'Fresno', 'Oakland', 'Chattanooga', 'Providence', 'Anchorage', 'Tucson', 'Minneapolis', 'Reno', 'Toledo', 'Greensboro', 'Canton', 'Las Vegas', 'Nashville', 'Oklahoma City', 'Madison', 'Newark', 'Louisville', 'St. Petersburg', 'Moreno Valley', 'Tampa', 'Norfolk', 'Washington, DC', 'Orlando', 'Virginia Beach', 'Tulsa']

    doc = nlp(text)

    for ent in doc.ents:
        if ent.label_ in labels or ent.text in cities_states:
            text = text.replace(ent.text, '')

    return text


def normalize_corpus(corpus, html_stripping=True, contraction_expansion=True,
                     accented_char_removal=True, text_lower_case=True, 
                     text_stemming=False, text_lemmatization=True, 
                     special_char_removal=True, remove_digits=True,
                     stopword_removal=True, stopwords=stopword_list):
    
    normalized_corpus = []
    # normalize each document in the corpus
    for doc in corpus:

        # strip HTML
        if html_stripping:
            doc = strip_html_tags(doc)

        # remove extra newlines
        doc = doc.translate(doc.maketrans("\n\t\r", "   "))

        # remove accented characters
        if accented_char_removal:
            doc = remove_accented_chars(doc)

        # expand contractions    
        if contraction_expansion:
            doc = expand_contractions(doc)

        # lemmatize text
        if text_lemmatization:
            doc = lemmatize_text(doc)

        # stem text
        if text_stemming and not text_lemmatization:
        	doc = simple_porter_stemming(doc)

        # remove special characters and\or digits    
        if special_char_removal:
            # insert spaces between special characters to isolate them    
            special_char_pattern = re.compile(r'([{.(-)!}])')
            doc = special_char_pattern.sub(" \\1 ", doc)
            doc = remove_special_characters(doc, remove_digits=remove_digits)  

        # remove extra whitespace
        doc = re.sub(' +', ' ', doc)

         # lowercase the text    
        if text_lower_case:
            doc = doc.lower()

        # remove stopwords
        if stopword_removal:
            doc = remove_stopwords(doc, is_lower_case=text_lower_case, stopwords=stopwords)

        # remove extra whitespace
        doc = re.sub(' +', ' ', doc)
        doc = doc.strip()
            
        normalized_corpus.append(doc)
        
    return normalized_corpus

[nltk_data] Downloading package stopwords to /home/Fazil/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


#### Add the cleaned text to the structure you created.


In [16]:
directory = 'smartcity'
filenames = []

# create a list of file names
for filename in os.listdir(directory):
    if filename.endswith(".pdf"):
        filenames.append(filename)

# go through each filename in dictionary and normalize the text
for filename in filenames:
    files_data[filename] = normalize_corpus(files_data[filename])

/tmp/ipykernel_34469/2210668626.py:20: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")


KeyboardInterrupt: 

### Clean Up: Discussion
Answer the questions below.

#### Which Smart City applicants did you remove? What issues did you see with the documents?

[Your Answer Here]

#### Explain what additional text processing methods you used and why.

I used labels to remove Cities and States from the text. I created a list of all the cities used in smartcity/ and used that to remove those names from the text too.

#### Did you identify any potientally problematic words?

[Your Answer Here]

## Experimenting with Clustering Models (Required)

Now, you'll start to explore models to find the optimal clustering model. In this section, you'll explore [K-means](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html), [Hierarchical](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.AgglomerativeClustering.html), and [DBSCAN](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html#sklearn.cluster.DBSCAN) clustering algorithms.
Create these algorithms with k_clusters for K-means and Hierarchical.
For each cell in the table provide the [Silhouette score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.silhouette_score.html#sklearn.metrics.silhouette_score), [Calinski and Harabasz score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.calinski_harabasz_score.html#sklearn.metrics.calinski_harabasz_score), and [Davies-Bouldin score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.davies_bouldin_score.html#sklearn.metrics.davies_bouldin_score).

In each cell, create an array to store the values.
For example, 

|Algorithm| k = 9 | k = 18| k = 36 | Optimal k| 
|--|--|--|--|--|
|K-means| [S,CH,DB]| [S,CH,DB] | [S,CH,DB] | [S,CH,DB] |
|Hierarchical |[S,CH,DB]| [S,CH,DB]| [S,CH,DB] | [S,CH,DB]|
|DBSCAN | X | X | X | [S,CH,DB] |



### Optimality 
You will need to find the optimal k for K-means and Hierarchical algorithms.
Find the optimality for k in the range 2 to 50.
Provide the code used to generate the optimal k and provide justification for your approach.


|Algorithm| k = 9 | k = 18| k = 36 | Optimal k| 
|--|--|--|--|--|
|K-means|--|--|--|--|
|Hierarchical |--|--|--|--|
|DBSCAN | X | X | X | -- |



#### How did you approach finding the optimal k?

[Your answer here]

#### What algorithm do you believe is the best? Why?

[Your Answer]

### Add Cluster ID to output file
In your data structure, add the cluster id for each smart city respectively. Show the to append the clusterid code below.

### Save Model

After finding the best model, it is desirable to have a way to persist the model for future use without having to retrain. Save the model using [model persistance](https://scikit-learn.org/stable/model_persistence.html). This model should be saved in the same directory as this notebook and should be loaded as the model for your `project3.py`.

Save the model as `model.pkl`. You do not have to use pickle, but be sure to save the persistance using one of the methods listed in the link.

## Derving Themes and Concepts (Required)

Perform Topic Modeling on the cleaned data. Provide the top five words for `TOPIC_NUM = Best_k` as defined in the section above. Feel free to reference [Chapter 6](https://github.com/dipanjanS/text-analytics-with-python/tree/master/New-Second-Edition/Ch06%20-%20Text%20Summarization%20and%20Topic%20Models) for more information on Topic Modeling and Summarization.

### Extract themes
Write a theme for each topic (atleast a sentence each).

[Your Answer]

[Your Answer]

[Your Answer]

### Add Topid ID to output file
Add the top two topics for each smart city to the data structure.

## Gathering Applicant Summaries and Keywords (Extra Credit Section)

For each smart city applicant, gather a summary and keywords that are important to that document. Gensim is outdated; try a spacy or nltk method.



### Add Summaries and Keywords
Add summary and keywords to output file.

## Write output data (Required)

The output data should be written as a TSV file.
You can use `to_csv` method from Pandas for this if you are using a DataFrame.

`Syntax: df.to_csv('file.tsv', sep = '')` \
`df.to_csv('smartcity_eda.tsv', sep='\t')`

# Moving Forward
Now that you have explored the dataset, take the important features and functions to create your `project3.py`.
Please refer to the project spec for more guidance.
